## Calculate regional mean transit times for each transit time map in the dataset
These will be used to filter which time points to look at in the time series analysis as we are only interested in PLDs > ATT. Anterior cingulate cortex, insular cortex, caudate, and putamen all show short transit times, making them good regions to maximize the number of data points we have to use. 

In [1]:
import os
import sys
import pandas as pd
import subprocess

local_dir = '../data/subjects'
roi_dir = '../rois'
hocpal = os.path.join(roi_dir, 'hocpal.nii.gz')
hospa = os.path.join(roi_dir, 'hospa.nii.gz')
gm_mask = os.path.join(roi_dir,'gm_resample_thr.nii.gz')

In [2]:
def refine_list(lst):
    '''
    creates a usable list of floats from the fsl output string

    :param lst: fsl output string
    :return: list of floats
    
    '''
    result_list = str(lst).split('\\n')
    result_list[0] = result_list[0].replace("b'", "")
    result_list.pop()
    result_list = [float(e) for e in result_list]
    
    return(result_list)
    

In [3]:
cols = ['session_id', 'gray_matter', 'left_acc', 'right_acc', 'left_insula', 'right_insula', 'left_caudate', 'right_caudate', 'left_putamen', 'right_putamen']
df = pd.DataFrame(columns=cols)
for root, dirs, files in os.walk(local_dir, topdown=False):
    tt_maps = [f for f in files if 'MNI152_tt' in f]
    if len(tt_maps) > 0:
        for tt_map in tt_maps:
            tt_map_path = os.path.join(root, tt_map)
            cmd1="fslstats -K '{}' '{}' -M".format(hocpal, tt_map_path)
            result1 = subprocess.check_output(cmd1, shell=True)
            cmd2="fslstats -K '{}' '{}' -M".format(hospa, tt_map_path)
            result2 = subprocess.check_output(cmd2, shell=True)
            cmd3="fslstats '{}' -k '{}' -M".format(tt_map_path, gm_mask)
            result3 = subprocess.check_output(cmd3, shell=True)
           
            # refine list
            result_list1 = refine_list(result1)
            result_list2 = refine_list(result2)
            # convert bytes-> str w no non-numeric or decimal points->float
            s = str(result3)
            s = ''.join(i for i in s if i.isdigit() or i=='.')
            result3=float(s)
            
            gray_matter = result3
            left_acc = result_list1[56]
            right_acc = result_list1[57]
            left_insula = result_list1[2]
            right_insula = result_list1[3] 
            left_caudate = result_list2[4]
            right_caudate = result_list2[15]
            left_putamen = result_list2[5]
            right_putamen = result_list2[16]
                        
            # get session name from file name
            ses_name = tt_map.split('_MNI152')[0]
            
            df.loc[len(df.index)] = [ses_name, gray_matter, left_acc, right_acc, left_insula, right_insula, left_caudate, right_caudate, left_putamen, right_putamen]

df.head()

,session_id,gray_matter,left_acc,right_acc,left_insula,right_insula,left_caudate,right_caudate,left_putamen,right_putamen
0,sub-HC010_ses-MR_FOLLOWUP,1483.561041,1363.563647,1433.913143,1282.433504,1216.445489,1148.279826,1584.002373,1221.612555,1563.332277
1,sub-HC010_ses-MR_BASELINE,1639.030927,1469.358166,1610.613272,1356.177378,1279.197735,1170.029530,1783.719164,1301.139330,1784.103641
2,sub-HC017_ses-MR_BASELINE,1294.992171,1113.530928,1153.863350,1072.964661,1072.261938,1024.305703,1358.307450,1107.343105,1338.017644
3,sub-HC017_ses-MR_FOLLOWUP,1418.859954,1201.335432,1230.490006,1130.319983,1147.456733,1100.572329,1476.850020,1207.943816,1454.020452
4,sub-HC028_ses-MR_BASELINE,1626.938207,1306.537493,1502.210526,1285.688587,1276.993436,1396.362445,1689.853116,1450.389997,1707.331128


In [4]:
df.to_csv('regionals_tts2.csv', index=False)

In [5]:
df

,session_id,gray_matter,left_acc,right_acc,left_insula,right_insula,left_caudate,right_caudate,left_putamen,right_putamen
0,sub-HC010_ses-MR_FOLLOWUP,1483.561041,1363.563647,1433.913143,1282.433504,1216.445489,1148.279826,1584.002373,1221.612555,1563.332277
1,sub-HC010_ses-MR_BASELINE,1639.030927,1469.358166,1610.613272,1356.177378,1279.197735,1170.029530,1783.719164,1301.139330,1784.103641
2,sub-HC017_ses-MR_BASELINE,1294.992171,1113.530928,1153.863350,1072.964661,1072.261938,1024.305703,1358.307450,1107.343105,1338.017644
3,sub-HC017_ses-MR_FOLLOWUP,1418.859954,1201.335432,1230.490006,1130.319983,1147.456733,1100.572329,1476.850020,1207.943816,1454.020452
4,sub-HC028_ses-MR_BASELINE,1626.938207,1306.537493,1502.210526,1285.688587,1276.993436,1396.362445,1689.853116,1450.389997,1707.331128
5,sub-HC028_ses-MR_FOLLOWUP,1471.643128,1238.825415,1441.235429,1179.403887,1220.765165,1291.568613,1606.994475,1317.900940,1673.812670
6,sub-HC021_ses-MR_BASELINE,1674.458227,1885.564396,1752.977169,1374.394693,1361.079525,1427.298413,1798.921630,1495.891096,1788.172727
7,sub-HC021_ses-MR_FOLLOWUP,1793.280349,1720.985109,1679.932494,1357.990079,1404.397273,1524.748148,1959.006284,1582.770427,1879.992815
8,sub-HC026_ses-MR_FOLLOWUP,1388.452827,1129.514647,1268.135661,1188.853351,1174.804570,1142.467057,1266.107085,1164.206228,1509.754730
9,sub-HC026_ses-MR_BASELINE,1332.120470,1111.284156,1152.804000,1189.720343,1102.374590,1091.566940,1290.969853,1102.197560,1370.505812


In [6]:
df.max()

session_id       sub-HC028_ses-MR_FOLLOWUP
gray_matter                    1793.280349
left_acc                       1885.564396
right_acc                      1752.977169
left_insula                    1533.890533
right_insula                   1404.397273
left_caudate                   1524.748148
right_caudate                  1959.006284
left_putamen                   1624.468193
right_putamen                   1951.27004
dtype: object